In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [ ]:
df = pd.read_csv('data/StudentsPerformance.csv')

In [ ]:
df.head()

In [ ]:
X = df.drop('math score', axis=1)

In [ ]:
X.head()

In [ ]:
print('categories in Gender:', end="")
print(df['gender'].unique())

In [ ]:
y = df['math score']

In [ ]:
y

In [ ]:
num_var = X.select_dtypes(exclude="object").columns
cat_var = X.select_dtypes(include="object").columns

In [ ]:
ohEncoder = OneHotEncoder()
scScaler= StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", ohEncoder, cat_var),
         ("StandardScaler", scScaler, num_var),        
    ]
)

In [ ]:
X = preprocessor.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

In [ ]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

In [ ]:
results =[]
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
 
    

    print(f"Model: {name}")
    print(f"R2 score: {r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    r2 = r2_score(y_test, y_pred)
    print("-" * 50)
    print("\n")

    results.append({'model Name': name,
                'R2': r2,
                "Mean Absolute Error (MAE)": mae,
                "Mean Squared Error (MSE)": mse,
                "Root Mean Squared Error (RMSE)":rmse
                })

In [ ]:
results =[]
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
 
    

    print(f"Model: {name}")
    print(f"R2 score: {r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    r2 = r2_score(y_test, y_pred)
    print("-" * 50)
    print("\n")

    results.append({'model Name': name,
                'R2': r2,
                "Mean Absolute Error (MAE)": mae,
                "Mean Squared Error (MSE)": mse,
                "Root Mean Squared Error (RMSE)":rmse
                })

In [ ]:
results =[]
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)


    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
 
    

    print(f"Model: {name}")
    print(f"R2 score: {r2:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    r2 = r2_score(y_test, y_pred)
    print("-" * 50)
    print("\n")

    results.append({'model Name': name,
                'R2': r2,
                "Mean Absolute Error (MAE)": mae,
                "Mean Squared Error (MSE)": mse,
                "Root Mean Squared Error (RMSE)":rmse
                })

In [ ]:
results_df = pd.DataFrame(results)
results_df